### 1. Đọc tập dữ liệu
- Ta bắt đầu bằng việc tạo một tập dữ liệu lưu trong file csv ../data/house_tiny.csv 
- Dữ liệu lưu ở các định dạng khác cũng có thể được xử lý tương tự.
- Hàm mkdir_if_not_exist dưới đây để đảm bảo mục ../data tồn tại

In [12]:
import os

def mkdir_if_not_exist(path):
    if not isinstance(path, str):
        path = os.path.join(*path)
    if not os.path.exists(path):
        os.makedirs(path)

Sau đây ta ghi tệp dữ liệu vào file csv theo từng hàng một

In [13]:
data_file = '../data/house_tiny.csv'
os.makedirs(os.path.dirname(data_file), exist_ok=True)  # Create the directory

# mkdir_if_not_exist('../data')
with open(data_file, 'w') as f:
    f.write('NumRooms, Alley, Price\n') # Collumn names
    f.write('NA,Pave,127500\n')  # Each row is a data point
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

Để nạp tập dữ liệu từ tệp csv vừa được tạo ra, ta dùng gói thư viện pandas và gọi hàm read_csv. Bộ dữ liệu này có 4 hàng và 3  cột, trong đó mỗi hàng biểu thị số phòng ("NumROooms"), kiểu lối đi ("Alley") và giá ("Price") của căn nhà.

In [14]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms  Alley   Price
0       NaN   Pave  127500
1       2.0    NaN  106000
2       4.0    NaN  178100
3       NaN    NaN  140000


### 2. Xử lý dữ liệu thiếu
- Với giá trị NaN là các giá trị bị thiếu, để xử lý dữ liệu thiếu, các cách thường được áp dụng là quy buộc (imputation) và xóa bỏ (deletion):
    + Quy buộc: Thay thế giá trị bị thiếu bằng giá trị khác
    + Xóa bỏ: Bỏ qua các giá trị bị thiếu
- Ta tách data thành input tương ứng với hai cột đầu và output tương ứng với cuột cuối cùng.
- Với các giá trị số bị thiếu trong input thì ta thay thế phần tử NaN bằng giá trị trung bình cộng của cột đó.

In [15]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, -1]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  Alley
0       3.0   Pave
1       2.0    NaN
2       4.0    NaN
3       3.0    NaN


Với các giá trị dạng hạng mục hoặc số rời rạc trong inputs ta coi "NaN" là một mục riêng, Vì cột Alley chỉ nhận hai giá trị riêng lẻ là "Pave" hoặc "NaN", pandas có thể tự động chuyển cột này thành 2 cột "Alley_Pave" và "Alley_nan". 
- Những hàng có Alley = Pave thì sẽ có Alley_Pave = 1, Alley_nan = 0
- Những hàng có Alley = NaN thì sẽ có Alley_Pave = 0, Alley_nan = 1

In [16]:
inputs = pd.get_dummies(inputs, dummy_na= True)
print(inputs)
inputs

   NumRooms   Alley_Pave   Alley_nan
0       3.0            1           0
1       2.0            0           1
2       4.0            0           1
3       3.0            0           1


,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


### 3. Chuyển sang định dạng ndarray
- với việc toàn bộ các giá trị trong input và output đều ở dạng số, ta có thể chuyển sang định dạng ndarray. Khi đã ở định dạng này, ta có thể biến đổi và xử lý chúng với những chức năng của ndarray.

In [17]:
from mxnet import np

x, y = np.array(inputs.values), np.array(outputs.values)
x, y

(array([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]], dtype=float64),
 array([127500, 106000, 178100, 140000], dtype=int64))